# 回环检测(Loop detection)

## 一、定义
在SLAM中前端进行特征点的提取和获得轨迹、地图的初值；而后端则负责对这些数据进行优化。然而由于VIO仅考虑了相邻关键帧，因此容易导致SLAM系统出现累积误差，而回环检测就是通过对智能体轨迹回环的判断来减少累积误差。

回环检测有基于里程计(Odometry based)和基于外观(Appearance based)两种类型，基于里程计的回环检测会根据前端VIO的地图推测此时运动轨迹是否回环。基于外观的回环检测与前端、后端无关，是一个独立的模块，通过两幅图像的相似性确定回环检测关系。基于外观的回环检测是目前的主流，以下讨论的是基于外观的回环检测算法。

回环检测是一个二分类问题，需要判断“两张图片是否在同一地方拍摄”，因此一般用准确率和召回率判断回环检测算法的好坏。在SLAM系统中倾向于**高准确率**的回环检测算法(降低FP)，这样才能保证地图的准确性。

## 二、词袋模型
### 2.1 介绍
基于外观的回环检测算法的关键是图像间的**相似性测度**，一种自然的想法是类似VIO提取特征点、匹配，匹配数量大于一定值则认为两张图像相似出现了回环。然而特征的匹配比较费时，且当光照变化时特征描述可能不稳定。回环检测采用的是词袋模型。

词袋(Bag-of-Words BoW)模型分为三步：
1. 确定基本概念，对应BoW的单词(Word),所有单词放在一起组成字典(Dictionary)。
2. 确定图像中出现了哪些字典中定义的概念，用单词出现的情况(二进制或直方图)描述图像，进而将图像转换成向量描述。
3. 比较不同图像的描述向量，比如二进制情况下式表示相似性测度(越接近1越相似)
$$s(a,b) = 1 - \frac{1}{W}||a - b||_1$$

可以看到，词袋模型不同于对特征点进行描述，转为对整幅图像进行描述，**描述子是图像在特征空间上以单词为基向量的坐标**。之所以说是词袋模型，是因为该模型对图像的描述与“单词”在图像的位置无关，这种性质也会带来一些问题，对词袋模型来说“眼睛在鼻子下面也是人脸”。

### 2.2 字典
下面介绍词袋模型中，字典的生成方法。字典是单词的集合，一个单词代表一个概念，而概念是从多幅图像的特征点上“抽象”出来的。因此字典生成问题类似**聚类**问题(常用Kmeans算法)，聚类获得的聚类中心就是单词。对于新图像的特征描述子，取最接近的聚类中心作为对应单词。

字典中单词可能有上万个，不利于单词的检索，需要借助于数据结构。一个常见的做法是用一个KD-tree
1. 在根节点用Kmeans把所有样本聚成k类。
2. 对于第一层的每个节点，把属于该节点的样本再聚成k类得到下一层。
3. 以此类推，最后得到叶子节点作为单词。

一个d层的KD-tree可以容纳$k^d$个单词，但查找单词仅需d次，复杂度$O(logN)$。

### 2.3 相似度计算
获得了字典之后，另一个重要的问题是如何进行表示可以更好的反应图像相似性，比如用0-1编码单词不出现、出现，但这样做的坏处是有些单词是常见的，而更有判别性的单词应该是少见的，因此需要对单词进行加权。

TF-IDF(Term Frequency-Inverse Document Frequency)频率-逆文档频率是文本检索中一种常用的加权方法，可以用于词袋模型中。具体做法是
1. 在建立字典时计算IDF，统计某个叶子节点中的特征数量相对于所有特征的比例，单词的IDF定义为：
$$IDF_i = log(\frac{n}{n_i})$$
2. TF指某个特征在单幅图像中出现的频率，假设图像A中单词w_i出现了n_i次，一共出现n个单词，则TF为
$$TF_i = \frac{n_i}{n}$$
3. 单词i
的权重等于TF与IDF的乘积
$$\eta_i = TF_i \times IDF_i$$
4. 图像的表示为  $\vec{v} = (\eta_1, \eta_2, ..., \eta_W)$ 

有了良好的表示后，可以用$L_1, L_2$范数定义图像的相似性测度。

## 三、总结
在检测到图像相似后，往往还需对回环进行验证，具体有两种方法。一种是时间一致性检测，即构造回环的缓存机制，认为单词检测到的回环不足以构成良好约束，在一段时间检测到的才是正确的回环。另一种是空间一致性检测，即对回环检测到的两帧进行特征匹配，估计相机运动，把运动放在位姿图中，检测与之前估计的出入。

回环检测是一个分类问题，因此与机器学习也有千丝万缕的联系。比如图像相似性测度问题就可以用端到端的度量学习替代。

## 四、参考资料
[1] 视觉SLAM十四讲-理论到实践 第十一章